# Class 9 - 26.5.19

# Testing and Test-Driven Development

## Introduction

We've already met one widely adopted programming technique which isn't used as much in the academy - object-oriented programming. Another such technique, ubiquitously used wherever code is written _outside_ the academy, is testing. In my opinion, this is a crucial topic in software development that isn't treated with the respect it should in the academy.

Sadly there are many examples of scientific code which failed. One of them is [this article](https://link.springer.com/article/10.3758/s13423-018-1489-7) from 2018, which checks how a dynamic image of a circle affects the understanding of speech, employed many modern and open-science techniques, as described [here](https://elemental.medium.com/when-science-needs-self-correcting-a130eacb4235) - pre-registration, [data availablity, source code availability](https://osf.io/bnvea/) and even peer replication. Unfortunately - the article as published in 2018 is completely wrong. The original conclusion was that these dynamic circles improved the understanding of what was said, while in truth they worsened it. The fault lied in a very simple mistake in the source code, still visible due to the author's comment (line ~25 in the link above), which resulted in opposite results.

Looking at the code after the correction, every half-decent programmers would immediately say that this piece of source code would never be accepted to any code base in any company. As a matter of fact, it looks just like an example of how to __not__ write code, just before some teacher goes over the file and starts correcting _everything_. Let's examine the flaws in it (P.S. I know it's in R, but this specific example is so "elegant" that I couldn't let it slip):

1. Everything is one large script. 
2. Variables aren't named carefully. 
3. Too many comments, zero functions.
4. Lots of commented out sections.

I won't start correcting it in front of your eyes, although I really want to, and instead we'll focus on one key aspect that's missing from this code, and from nearly all scientific code, which would've caught this error prematurely. This aspect or technique is of course testing.

Tests are (usually) short pieces of code designed to assert that a small portion of your program does what you intend it to do. 

For example, if I have a class designed to perform calculations on some DataFrame that was created somewhere else, perhaps by adding some of its columns together, averaging them out and displaying the result, then I wish for my code to be correct and deterministic, in a sense that a single, defined input will always give the same correct output.

This isn't trivial even for the most basic functions in Python. Let's look at a simple normalization function:

In [16]:
import numpy as np

In [17]:
def normalize_array(a):
    """Normalizes array"""
    minimum = min(a)
    no_min = a - minimum
    return no_min / max(no_min)

Since we're so smart we'll even check to see if it works!

In [15]:
normalize_array(np.array([1, 2, 3]))

array([0. , 0.5, 1. ])

In [14]:
normalize_array(np.array([0, 100, 250]))

array([0. , 0.4, 1. ])

Amazing! We can move on to our next function.

...

...


...Or can we?

What issues does this function have? We'll start from the more basic ones, and slowly dive deeper.

1. The docstring is lacking, to say the least.
2. Variable names are awful.
3. It uses the Pythonic `min` and `max` functions instead of numpy's `min` and `max`. This will lead to a reduction in performance, but also it has different behavior for certain inputs.
4. Many different inputs values will raise warning, exceptions, or worse - won't raise any error and simply return the wrong result.
5. It doesn't take data types into consideration.
6. This functionality exists in a few pretty popular libraries - why not use them instead of re-inventing the wheel?

Now, you might have written this function differently - perhaps you've remembered to use numpy's version for `min` and `max`, and perhaps you weren't so lazy and managed to write a decent docstring, but I'm pretty sure that most of you wouldn't have taken all of the points above into consideration when writing this function.

The importance of this function is also pretty clear - it's probably located somewhere in the heart of our data processing pipeline, and if there's a chance it will return the wrong result sometimes, then our resulting computation is in jeopardy. The only I know of solving this issue is by writing tests, so let's do that.

You've already seen unit tests in your homework assignments. You were asked to make sure that your solution passes all tests before submitting. This exemplifies a way through which we can be more certain that our program does what we thought it was doing.

Tests are important to us for two reasons. The first one is that even simple programs are more complicated than we think. The normalization function is a good example, but you might imagine that as soon as we add interfaces between classes, methods and functions, things might get a bit messy. For example, in the aviation industry, for each line of code a software has, you may find about 8 lines dedicated to testing it.

Moreover, when we deal with user input - data files, parameters for script - we should expect the unexpected, even if the main user is ourselves. Our future self in a few months will probably not remember the type of every parameter it has to enter.

The second reason is Python's dynamic nature, or _duck-typing._ If you want to enforce a function to only accept inputs of a single type, _you_ must be the one writing these assertions, either outside or inside the function. For example, a function that adds two numbers needs a `isinstance(value, (int, float))` somewhere near its top to avoid these mistakes. Statically-typed languages, like C, define a type for each variable. A function adding two integers simply cannot accept a non-integer input. 

Python's dynamic nature is a blessing on many occasions, but it can sometimes be a real pain. This nature is the second important reason to write tests to our code. Many cases that in other programming languages would've resulted in a simple `TypeError`, can cause major bugs in Python due to wrong input types.

## Solution - Test your code

As you see, I want you to write tests. Let's examine the iterative process that produces code and tests. We'll go back to the normalization example later, we'll start with a more simple function. Let's assume I have two positive integer number I wish to add together - perhaps they're the result of some complicated pipeline which means that I'm not entirely sure that I will indeed have two positive integer numbers when they enter this function.

How does this function look like?

In [18]:
def intadd(num1: int, num2: int) -> int:
    """Adds two numbers, assuming they're both positive integers.
    
    Parameters
    ----------
    num1, num2 : int
        Positive integers
    
    Returns
    -------
    int
        Resulting positive integer
    
    Raises
    ------
    ValueError
        If either number is negative 
    """
    if (num1 < 0) or (num2 < 0):
        raise ValueError(f"Received {num1, num2}; expected positive integers")
    return num1 + num2

That's a good start! We defined the inputs, output type and even added a check that sees that the numbers are indeed positive. The next step, after I have a working skeleton of my implementation, is to test it. To do so I'll start writing a few tests:

In [43]:
# Test functions always start with "test_..."
def test_basic_addition():
    assert intadd(2, 4) == 6

    
def large_number_additions():
    first = 1_000_000
    second = 2_000_001
    result = first + second
    assert intadd(first, second) == result

Above you see an example of using assertion in tests, which work hand-in-hand with `pytest`, the library we've been using to run our HW tests. Pytest looks for these asserts and executes them in a smart manner, which helps us in writing very concise and clear tests. 

To run these tests we need to use the `pytest` command from the command line, or use VSCode's built in pytest integration (that flask icon from a few weeks ago). Let's do it from VSCode, since it's a bit harder to do so straight from the command line. The tests are in the "tests_demo" folder.

### [[Tests demo]]

After running these two simple tests we see they pass, which is good. But we also coded another functionality to our `intadd` function - the way it raises a `ValueError` when it encounters negative numbers. We would like to verify that it indeed raises an exception (and the correct one) when it encounters these illegal inputs.

In [1]:
import pytest


def test_negative_inp1():
    with pytest.raises(ValueError):
        result = intadd(-1, 1)


def test_negative_inp2():
    with pytest.raises(ValueError):
        result = intadd(-1, 1)

Again, pytest excels at brevity and readability - we use a context manager (`with` statement) to verify that calling the function with these illegal inputs indeed raises the correction exception.

During the time it tooks us to write the original `intadd` implementation and to test it, we thought of another case - what if the inputs are floating point numbers? That's also illegal, and it may tell us that there's a problem with our computation which resulted in a float instead of an integer. Let's add this functionality and tests together.

In [ ]:
def intadd(num1: int, num2: int) -> int:
    """Adds two numbers, assuming they're both positive integers.
    
    Parameters
    ----------
    num1, num2 : int
        Positive integers
    
    Returns
    -------
    int
        Resulting positive integer
    
    Raises
    ------
    ValueError
        If either number is negative
    TypeError
        If either number isn't an integer
    """
    if (num1 < 0) or (num2 < 0):
        raise ValueError(f"Received {num1, num2}; expected positive integers")
    if (isinstance(num1, float)) or (isinstance(num2, float)):
        raise TypeError(f"Received {num1, num2}; expected integers, not floats")
    return num1 + num2

In [11]:
def test_float_inp():
    with pytest.raises(TypeError):
        intadd(1.0, 1)
    with pytest.raises(TypeError):
        intadd(1, 1.0)

Great, we have floating points covered. When the test cases are really simple and clearly connected, it's OK if we push to `assert` expressions into the same test. Otherwise it's always better to have only one `assert` per test, especially for these simple tests, which we also call _unit tests_.

We're almost all set - but again I thought of another edge case. We're testing floating points - but what if our input isn't a floating point number, and is also not an integer? The current tests only check for positive integers and floats, but the input could be strings, or None, or many other things. We need a more general test, and we should also switch between the `if` checks, so let's do that:

In [13]:
def intadd(num1: int, num2: int) -> int:
    """Adds two numbers, assuming they're both positive integers.
    
    Parameters
    ----------
    num1, num2 : int
        Positive integers
    
    Returns
    -------
    int
        Resulting positive integer
    
    Raises
    ------
    ValueError
        If either number is negative
    TypeError
        If either number isn't an integer
    """
    if (not isinstance(num1, int)) or (not isinstance(num2, int)):
        raise TypeError(f"Received {num1, num2}; expected integers, not {type(num1), type(num2)}")
    if (num1 < 0) or (num2 < 0):
        raise ValueError(f"Received {num1, num2}; expected positive integers")
    return num1 + num2

In [16]:
def test_str_input():
    with pytest.raises(TypeError):
        intadd('s', 0)

def test_none_input():
    with pytest.raises(TypeError):
        intadd(None, 0)

def test_list_input():
    with pytest.raises(TypeError):
        intadd([], 1)

We should probably add a few more cases, but it's getting tedious - the test itself is indeed short, but we're repeating ourselves a bunch, while the only things that changes is the input. 

Pytest is a great library, and luckily it allows us to paramaterize over the input, leaving the skeleton of the function intact. It uses a special syntax called _decorators_ which we haven't met yet. Until we discuss it, which will happen shortly, let's just use the feature without really understanding what's going on:

In [ ]:
typeerror_inputs = [
    ('s', 0), 
    (None, 0), 
    ([], 1), 
    ((,), 10), 
    ({}, 2), 
    ({1}, 20)
]

@pytest.mark.parametrize("inp1, inp2", typerror_inputs)
def test_invalid_input_raises_typeerror(inp1, inp2):
    with pytest.raises(TypeError):
        intadd(inp1, inp2)
        intadd(inp2, inp1)

The decorator function, starting with `@`, receives two inputs:
1. The names of all function arguments you wish to parametrize.
2. The arguments that should be given to the function, as a list of tuples, where each tuple has the length of the needed arguments.

Then pytest runs this function for all given inputs and asserts that the given condition holds. If one of them fails it wouldn't stop running the remaining inputs, but will point us at the specific input that didn't pass the test. Note that I'm calling `intadd` twice, once for each input ordering.

Here's both the final version of the function and the tests, just so we could see them together:

In [ ]:
def intadd(num1: int, num2: int) -> int:
    """Adds two numbers, assuming they're both positive integers.
    
    Parameters
    ----------
    num1, num2 : int
        Positive integers
    
    Returns
    -------
    int
        Resulting positive integer
    
    Raises
    ------
    ValueError
        If either number is negative
    TypeError
        If either number isn't an integer
    """
    if (not isinstance(num1, int)) or (not isinstance(num2, int)):
        raise TypeError(f"Received {num1, num2}; expected integers, not {type(num1), type(num2)}")
    if (num1 < 0) or (num2 < 0):
        raise ValueError(f"Received {num1, num2}; expected positive integers")
    return num1 + num2


standard_inputs = [(1, 2, 3), (100000, 200000, 100000 + 200000)]
@pytest.mark.parametrize('inp1, inp2, result', standard_inputs)
def test_valid_inputs(inp1, inp2, result):
    assert result == intadd(inp1, inp2)
    assert result == intadd(inp2, inp1)
    

negative_valueerror_inputs = [(-1, 1), (1, -2), (-2, -3)]
@pytest.mark.parametrize('inp1, inp2', negative_valueerror_inputs)
def test_negative_input_raises_valueerror(inp1, inp2):
    with pytest.raises(ValueError):
        intadd(inp1, inp2)
        intadd(inp2, inp1)

        
typeerror_inputs = [
    ('s', 0), 
    (None, 0), 
    ([], 1), 
    ((), 10), 
    ({}, 2), 
    ({1}, 20)
]

@pytest.mark.parametrize("inp1, inp2", typeerror_inputs)
def test_invalid_input_raises_typeerror(inp1, inp2):
    with pytest.raises(TypeError):
        intadd(inp1, inp2)
        intadd(inp2, inp1)

We could perhaps continue a bit more, but I think that the principle is clear. Before moving on to a more difficult example, let's examine a few more features of tests:

1. The test functions have no docstring, very long names, and a concise body. This allows them to be very readable, fast to execute, and very low chance that they themselves will contain any bugs.
2. Writing tests forces us to design an "API" of the function. API stands for application programming interface, and it's usually a property of large libraries, but here we mean that after writing the tests, the function can only output something meaningful only if you use it in a very strict way, which is great. This makes sure that bugs won't just spread throughout the execution, but will stop at some point.
3. There are many more lines of code of tests than of the actual function that's being tested. This is usually a sign of good coding practices.
4. Some people write the tests before the actual function, known as test-driven development (TDD). In this method you first think of edge cases and possible inputs, write a tests that fails, and then corrects the function until the test passes.
5. Pytest has a ton of other features which we won't cover. [Here](https://docs.pytest.org/en/latest/parametrize.html#basic-pytest-generate-tests-example) is one link for the basics, and [here](https://hackebrot.github.io/pytest-tricks/create_tests_via_parametrization/) you can find a clear blog post giving more information about parameterization.
6. Tests should be run as often as possible, to make sure that your changes aren't affecting the existing behavior of your code.
7. You should try very hard to 'translate' bugs into tests. These might be the most important tests you'll write.

### Back to the normalization example

In [17]:
def normalize_array(a):
    """Normalizes array"""
    minimum = min(a)
    no_min = a - minimum
    return no_min / max(no_min)

Above we have the previous normalization function that we wrote. It's less obvious how to test it due to a couple of reasons:

1. It's very hard to calculate by hand the correct result for any arbitrary input, since the mathematical operation here isn't as trivial as in the previous `intadd` example.
2. It's not as easy to create the inputs, and what exactly are the allowed input shapes and types?

As I said before, we probably shouldn't write it ourselves since smarter people already wrote it in great libraries such as `scikit-learn`. Let's just look at their final implementation and tests. Note that their function can work with arbitrary normalization ranges, not only (0, 1), but it's the same idea. [Here's](https://github.com/scikit-learn/scikit-learn/blob/483cd3eaa/sklearn/preprocessing/_data.py#L200) the class and [here](https://github.com/scikit-learn/scikit-learn/blob/483cd3eaa3c636a57ebb0dc4765531183b274df0/sklearn/preprocessing/tests/test_data.py#L731) are its tests. It's clear that a lot of effort was put into writing and testing this function, so our humble attempt will not be as complete as theirs, but the exercise is useful nonetheless.

To start off our attempt we'll re-write the function itself using 100% numpy-based code:

In [24]:
def normalize_array(arr: np.ndarray) -> np.ndarray:
    """Normalizes an array to the range [0, 1].
    
    The maximal element will be 1 and the minimal 0, while the 
    scale remains the same.
    
    Paramaters
    ----------
    arr : np.ndarray
        Input array to normalize
    
    Returns
    -------
    np.ndarray
        Normalized array
    """
    arr -= arr.min()
    return arr / arr.max()

The best way to know if it's working is not to run it, but to run it inside a test environment. Let's do that:

In [29]:
def test_norm_noop():
    inp = np.array([0., 0.5, 1.])
    result = normalize_array(inp)
    truth = inp.copy()
    np.testing.assert_array_equal(truth, result)
    
test_norm_noop()

A couple of things to note:
1. We specifically wrote an array that should remain exactly the same, this is the easiest example.
2. To test array equality we actually want to traverse the array and test that each element is identical to its partner. We also want to make sure that the shape and data types are identical. Numpy helps with that using its built-in `testing` module.
3. Floating point arrays may differ by a tiny bit (on the order of 1e-6) and still be considered equal for many use cases. In the above function we're asserting true equality, but often times (like below) we'll use `test_array_almost_equal` instead.

What else can we do? Let's write a few more tests:

In [36]:
def test_norm_simple():
    inp = np.array([0., 1., 2.])
    result = normalize_array(inp)
    truth = np.array([0, 0.5, 1])
    np.testing.assert_array_almost_equal(truth, result)
    

def test_norm_simple_negative():
    inp = np.array([0., -1., -2.])
    result = normalize_array(inp)
    truth = np.array([1, 0.5, 0])
    np.testing.assert_array_almost_equal(truth, result)
    
    
test_norm_simple()
test_norm_simple_negative()

Thinking about the possible case where the input is negative lead to us figuring the interface, or API, of our function. Meaning that we now know how it behave with (basic) negative inputs, and we also "documented" it by writing a test for it.

In the `intadd` example we had we could write pretty complex inputs and know that everything is working, but here it's a bit harder to "compute" by ourselves. We also don't have any exceptions we wish to raise. So how can we test a function too complex for us to know the "true" result?

What we'll do as a first step is to assert some properties of the resulting array. For example, the resulting arrays should be of floating point type (which float?), since our functions has division in it. We also know that the maximal value has to be 1, and the minimal - 0. We'd also like to conserve the shape of the input array. Let's add these tests:

In [44]:
@pytest.mark.parametrize('dt', [np.float32, np.float64])
def test_norm_dtype(dt):
    result = normalize_array(np.array([1., 2, 3], dtype=dt))
    assert result.dtype == dt

    
def test_norm_min_max():
    rand_arr = np.random.randint(0, 100, (100, 20))
    result = normalize_array(rand_arr)
    assert result.max() == 1.0
    assert result.min() == 0.0


shapes = [(1,), (10,), (1, 2), (5, 1), (20, 20, 30)]
@pytest.mark.parametrize('shape', shapes)
def test_norm_shapes(shape):
    rand_arr = np.random.randint(10, 20, shape)
    result = normalize_array(rand_arr)
    assert result.shape == shape

After running them with pytest we see we're OK. Now we should think of edge cases, i.e. inputs that could mean trouble. Three types of inputs come to mind: Negative numbers (which we already dealt with, maybe), zero, and NaNs. 

Let's write a test with 0 in the input and see what happens:

In [59]:
def test_norm_zero_is_max():
    arr = np.array([0, 0])
    result = normalize_array(arr)
    np.testing.assert_array_almost_equal(result, np.array([0, 0]))

We expected the test to return the original array since you can't really normalize an array made of zeros, but this clearly didn't happen. This is the first real bug of our function, and it also exposes a weakness in the API of that function - how should we deal with these types of inputs? Division by 0 is a big no no, and it's very possible that without tests we wouldn't have thought of such edge cases.

How can we correct this behavior? Currently, if the minimal value is 0 we'll have no issues, and if the maximal value is 0 we also won't have any issues, since we subtract the minimal value from the entire array which will change the maximal value. So let's add a check that see whether we're about to divide by zero.

In [61]:
def normalize_array(arr: np.ndarray) -> np.ndarray:
    """Normalizes an array to the range [0, 1].
    
    The maximal element will be 1 and the minimal 0, while the 
    scale remains the same.
    
    An array containing only identical values will be returned as a zeroed array.
    
    Paramaters
    ----------
    arr : np.ndarray
        Input array to normalize
    
    Returns
    -------
    np.ndarray
        Normalized array
    """
    arr -= arr.min()
    max_ = arr.max()
    return arr if np.isclose(max_, 0.) else arr / max_

We used a terneray operation to quickly express the new logic. As a reminder, this expression will return `arr` if `np.isclose(max_, 0.)` returns True and `arr / max_` if it returns False. The `np.isclose` function compares floating point numbers correctly.

How does the test run now?

In [62]:
def test_norm_zero_is_max():
    arr = np.array([0, 0])
    result = normalize_array(arr)
    np.testing.assert_array_almost_equal(result, np.array([0, 0]))


def test_norm_identical_is_zeroed():
    arr = np.array([10, 10, 10.], dtype=np.float32)
    result = normalize_array(arr)
    np.testing.assert_array_almost_equal(result, np.array([0., 0., 0.,], dtype=np.float32))

Great! Another potential bug squashed. 

One more special input type remains - NaNs. What do we expect to happen? There are three ways to deal with NaNs, as we know:

1. Using NaN-aware functions instead of the standard ones, and then NaNs can live happily with other values.
2. Replace them with a sentinel value (etc. 0, some large number).
3. Remove them altogether. 

Removing them (3) is usually not a good option, as it changes the shapes of the arrays. Replacing is an option, but it does change the statistics of the array, so it might not work for all cases. We can use NaN-aware functions, but they show degraded performance which is less-than-ideal many times. The "correct" answer really depends on the data - if we don't intend to calculate global statistics for it, like its total mean or STD, for example when we're working with EEG recordings - then it might be fine to simply replace them. Otherwise we'd have to use NaN-aware functions and leave the NaNs as-is. 

Due to the origin of our data we believe it's fine to simply replace NaNs, `inf`s and such with 0, so we'll add that line to our function and then write a couple more tests.

In [65]:
def normalize_array(arr: np.ndarray) -> np.ndarray:
    """Normalizes an array to the range [0, 1].
    
    The maximal element will be 1 and the minimal 0, while the 
    scale remains the same.
    
    Notes
    -----
    * An array containing only identical values will be returned as a zeroed array.
    * NaNs are automatically converted to 0's before computation.
    
    Paramaters
    ----------
    arr : np.ndarray
        Input array to normalize
    
    Returns
    -------
    np.ndarray
        Normalized array
    """
    arr = np.nan_to_num(arr)
    arr -= arr.min()
    max_ = arr.max()
    return arr if np.isclose(max_, 0.) else arr / max_


def test_norm_with_nan():
    arr = np.array([np.nan, 1])
    result = normalize_array(arr)
    truth = np.array([0, 1.])
    np.testing.assert_array_almost_equal(result, truth) 

def test_norm_all_nans():
    arr = np.array([[np.nan, np.nan]])
    result = normalize_array(arr)
    truth = np.array([[0., 0.]])
    np.testing.assert_array_almost_equal(result, truth)

In [66]:
test_norm_with_nan()
test_norm_all_nans()

One of the benefit of testing is that we're feeling very confident in our code. In this case I'm pretty sure that once all tests pass the function is doing what's I think it's doing. This security is rare, and if you feel secure about untested code you wrote then I believe this normalization example shows you the number of pitholes that you can fall into when writing "super-simple" scientific code.

If you think about it a bit more, I haven't __really__ tested the function for most inputs. I looked for edge cases and simple ones, but I didn't really do anything with the typical inputs I should actually handle. This should come at no surprise - I can't really manually normalize large arrays, so how could I get my "truth" array?

The way to get these "truth" inputs varies. In statistics code, for example, each new model they add to their libraries is verified with a few other implementations, to make sure that both SPSS, R and JMP output the same results given the same inputs. Simpler cases - like we have here - may create their ground truths by using "snapshot testing", which simply means taking a snapshot of the current output the code is giving, and making sure that new code we write doesn't give different results from the old code.

The process is simple: Run your code for some random input, and save both input and result. Then write a test that reads the input, runs the function on it and tests that the new result is identical to the one saved to disk.

In [75]:
# Let's create mock data
shape1 = (100, 100, 3)
input1 = np.random.randint(20, 100, shape1)
output1 = normalize_array(input1)

shape2 = (10, 4, 1, 3)
input2 = np.random.randn(*shape2) * 12
output2 = normalize_array(input2)

np.savez('tests_demo/normalize_array/snapshot_05_2020.npz', input1=input1, output1=output1, input2=input2, output2=output2)

In [76]:
def test_snapshots():
    fname = 'tests_demo/normalize_array/snapshot_05_2020.npz'
    data = np.load(fname)
    num_of_snaps = len(data) // 2
    for snap in range(1, num_of_snaps + 1):
        inp = data[f"input{snap}"]
        old_output = data[f"output{snap}"]
        new_output = normalize_array(inp)
        np.testing.assert_array_almost_equal(old_output, new_output)

At this point we can say we're done. 16 tests for this function seems about right. We can do better - we'll show a nice library in a couple of lessons - but I believe that the importance of tests came across.

### Integration Testing

Unit tests repeatedly tests functions or methods your write under different inputs, and are the backbone of any reliable test suit. However, unit tests are not enough, since they don't check the interface between the different functions and classes in your application.

Integration tests are larger, heavier tests that take at least two components, or units, of your application, and makes sure that they interact well with each other.

Obviously, if we start taking each two consecutive functions and write an integration test for that pair, and then continue with all three consecutive functions and write these integration tests, and so on - we'll never finish writing the damn application. That's why integration testing is used at crucial junctions of our application, between major classes for example.

## Exercise

Write a class that returns the difference between the Fibonacci series and the prime numbers series, up to some _n._ The output should be an array of numbers, n-items long, and a plot to accompany it. As an example, for $n=3$, I expect the array to be `np.array([-2, -2, -4])`. Write the class in a test-driven development style.

A reminder, the Fibonacci series is a series of numbers starting from (0, 1), with its next element being the sum of the previous two numbers: 0, 1, 1, 2, 3, 5, ... Prime numbers start from 2 and are only divisible by themselves and 1 without a remainder.

You decide on the class' interface and the details of implementation. You may use `numpy`, and I insist that you write at least 5 unit tests and 1 integration test for this class.

Don't try to implement things in a performant way, with fancy algorithms. The focus here is the unit tests and test-driven development.

### Exercise solutions are in the directory `fib_ser`